<a href="https://colab.research.google.com/github/AustinSopocko/Sentiment-Analysis/blob/main/Sentiment_Analysis_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IMPORTS

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sklearn
from sklearn.model_selection import train_test_split

import nltk
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.corpus import stopwords

import string

import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
#Reading file
dataframe = pd.read_csv("Training.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Training.csv'

In [ ]:
#Lowercase all text
dataframe['text'] = dataframe['text'].str.lower()

In [ ]:
#Func for replacing punctuations
def remove_punctuations(new_text):
  punctuations = string.punctuation
  return new_text.translate(str.maketrans(punctuations,' '*len(punctuations))) #new code

In [ ]:
#Apply removing punctuations function
dataframe['text'] = dataframe['text'].apply(lambda x: remove_punctuations(x))

In [ ]:
#Get and remove stopwords function
stopWords = set(stopwords.words('english'))
def remove_sw(text):
  return " ".join([word for word in text.split() if word not in stopWords])

In [ ]:
#Apply removing stopwords function
dataframe['text'] = dataframe['text'].apply(lambda x: remove_sw(x))

In [ ]:
#Initializing tokenizer, sequencing and padding data
tokenizer = Tokenizer(num_words = 10000)
tokenizer.fit_on_texts(dataframe['text'])
sequences = tokenizer.texts_to_sequences(dataframe['text'])
data = pad_sequences(sequences, maxlen=100)

In [ ]:
#Splitting data into train/test
x=data
y=dataframe['label']

x_train, x_test, y_train, y_test = train_test_split(
  x,y, random_state=104,test_size=0.20, shuffle=True)

In [ ]:
#Creating NN model
embedding_dim = 16

model = tf.keras.Sequential([
  layers.Embedding(10000, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1, activation='sigmoid')])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          160000    
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 160017 (625.07 KB)
Trainable params: 160017 (625.07 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
#Compiling model
model.compile(loss=losses.BinaryCrossentropy(),
              optimizer='Adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.5)])

In [ ]:
#Training model
epochs = 10
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = epochs)

Epoch 1/10
125/125 [==============================] - 2s 8ms/step - loss: 0.6894 - binary_accuracy: 0.6560 - val_loss: 0.6854 - val_binary_accuracy: 0.6890
Epoch 2/10
125/125 [==============================] - 1s 6ms/step - loss: 0.6718 - binary_accuracy: 0.7358 - val_loss: 0.6654 - val_binary_accuracy: 0.7320
Epoch 3/10
125/125 [==============================] - 1s 7ms/step - loss: 0.6369 - binary_accuracy: 0.8005 - val_loss: 0.6341 - val_binary_accuracy: 0.7670
Epoch 4/10
125/125 [==============================] - 1s 8ms/step - loss: 0.5912 - binary_accuracy: 0.8257 - val_loss: 0.5961 - val_binary_accuracy: 0.8040
Epoch 5/10
125/125 [==============================] - 1s 12ms/step - loss: 0.5391 - binary_accuracy: 0.8575 - val_loss: 0.5581 - val_binary_accuracy: 0.8180
Epoch 6/10
125/125 [==============================] - 1s 10ms/step - loss: 0.4904 - binary_accuracy: 0.8752 - val_loss: 0.5218 - val_binary_accuracy: 0.8310
Epoch 7/10
125/125 [==============================] - 1s 7ms/s

In [ ]:
#Testing model
loss, accuracy = model.evaluate(x_test, y_test)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

32/32 [==============================] - 0s 2ms/step - loss: 0.4247 - binary_accuracy: 0.8400
Loss:  0.42465320229530334
Accuracy:  0.8399999737739563
